In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

In [ ]:
import pandas as pd

In [ ]:
# Create the model
model = ConcreteModel()

# Inputs
num_dogs = range(1, 16) # indexed by [d]
num_volunteers = range(1, 6) #indexed by [v]

dog_time_required = {1: 33, 2: 38, 3: 33, 4: 29, 5: 53, 6: 23, 7: 43, 8: 27, 9: 55, 10: 56, 11: 24, 12: 35, 13: 55, 14: 41, 15: 59}
volunteer_available_time = {1: 300, 2: 300, 3: 300, 4: 300, 5: 300}

# Decision Variables
model.x = Var(num_dogs, num_volunteers, within=Binary)


# Objective
model.obj = Objective(expr=sum(dog_time_required[d] * model.x[d, v] for d in num_dogs for v in num_volunteers), sense=minimize)

# Constraints
# Each dog is walked by one volunteer
model.walk_once = ConstraintList()
for d in num_dogs:
    model.walk_once.add(sum(model.x[d, v] for v in num_volunteers) == 1)

# Each required exercise time for a dog is less than the volunteer's available time
model.volunteer_time = ConstraintList()
for v in num_volunteers:
    model.volunteer_time.add(sum(dog_time_required[d] * model.x[d, v] for d in num_dogs) <= volunteer_available_time[v])

# Each volunteer must walk at least one dog
model.walk_at_least_one_dog = ConstraintList()
for v in num_volunteers:
    model.walk_at_least_one_dog.add(sum(model.x[d, v] for d in num_dogs) >= 1)

# Each volunteer can only walk at most three dogs
model.walk_at_most_three_dogs = ConstraintList()
for v in num_volunteers:
    model.walk_at_most_three_dogs.add(sum(model.x[d, v] for d in num_dogs) <= 3)

In [ ]:
# Solve the model
opt = SolverFactory('cbc')
opt.options['seconds'] = 5
opt.options['ratioGap'] = 0
results = opt.solve(model, tee=False)

In [ ]:
# Display the results
print("Objective Value: ", model.obj())
print("\nVolunteer Assignments:")
for d in num_dogs:
    for v in num_volunteers:
        if value(model.x[d, v]) == 1:
            print(f"Dog {d} is assigned to Volunteer {v}")

Objective Value:  604.0

Volunteer Assignments:
Dog 1 is assigned to Volunteer 2
Dog 2 is assigned to Volunteer 4
Dog 3 is assigned to Volunteer 5
Dog 4 is assigned to Volunteer 2
Dog 5 is assigned to Volunteer 3
Dog 6 is assigned to Volunteer 1
Dog 7 is assigned to Volunteer 3
Dog 8 is assigned to Volunteer 4
Dog 9 is assigned to Volunteer 5
Dog 10 is assigned to Volunteer 4
Dog 11 is assigned to Volunteer 5
Dog 12 is assigned to Volunteer 1
Dog 13 is assigned to Volunteer 3
Dog 14 is assigned to Volunteer 2
Dog 15 is assigned to Volunteer 1
